In [ ]:
!pip install -U sentence-transformers rank_bm25 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.0 MB/s eta 0:00:00


In [ ]:
import json
import nltk
from nltk import sent_tokenize

nltk.download('punkt')
import time
from pprint import pprint
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Personalization/question_rewritten.zip -d ./

Archive:  /content/drive/MyDrive/Personalization/question_rewritten.zip
   creating: ./question_rewritten/
  inflating: ./question_rewritten/val_question_rewritten_1.csv  
  inflating: ./question_rewritten/test_question_rewritten_1.csv  
  inflating: ./question_rewritten/train_question_rewritten_1.csv  
  inflating: ./question_rewritten/.~lock.train_question_rewritten_1.csv#  


In [ ]:
!unzip /content/drive/MyDrive/Personalization/FoCus_modified.zip -d ./

Archive:  /content/drive/MyDrive/Personalization/FoCus_modified.zip
   creating: ./FoCus_modified/
  inflating: ./FoCus_modified/val_set.json  
  inflating: ./__MACOSX/FoCus_modified/._val_set.json  
  inflating: ./FoCus_modified/test_set.json  
  inflating: ./__MACOSX/FoCus_modified/._test_set.json  
  inflating: ./FoCus_modified/train_set.json  
  inflating: ./__MACOSX/FoCus_modified/._train_set.json  


In [ ]:
df = pd.read_csv("/content/question_rewritten/train_question_rewritten_1.csv")
df.columns

Index(['dialogID', 'utterance', 'query', 'answer', 'dialog_history',
       'ground_knowledge', 'knowledge', 'ground_persona',
       'question_rewritten'],
      dtype='object')

In [ ]:
hit_knowledge_df = pd.read_csv("/content/drive/MyDrive/UMBC/hit_knowledge/train_knowledge.csv")

In [ ]:
def get_paragraphs(knowledge):
  paragraphs = []
  for paragraph in knowledge.split("\n"):
      if len(paragraph.strip()) > 0:
          paragraphs.append(sent_tokenize(paragraph.strip()))
  return paragraphs

In [ ]:
len(df.iloc[0]['knowledge'])

12657

In [ ]:
test_json = "/content/FoCus_modified/train_set.json"

In [ ]:
with open(f"{test_json}") as f:
  test_data = json.load(f)

In [ ]:
test_data[0]['dialogID']

'CGHGPTDOA0O7'

In [ ]:
df.iloc[0]['dialogID']

'CGHGPTDOA0O7'

In [ ]:
def get_paragraphs(knowledge):
  paragraphs = []
  for paragraph in knowledge.split("\n"):
      if len(paragraph.strip()) > 0:
          paragraphs.append(sent_tokenize(paragraph.strip()))
  return paragraphs

In [ ]:
def get_passages(paragraphs, window_size=4):
  passages = []
  for paragraph in paragraphs:
      for start_idx in range(0, len(paragraph), window_size):
          end_idx = min(start_idx+window_size, len(paragraph))
          passages.append(" ".join(paragraph[start_idx:end_idx]))
  return passages

In [ ]:
def retrieve_knowledge(query, passages, corpus_embeddings, n_count = 3):
  # corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)
  question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  question_embedding = question_embedding.cuda()
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  hits = hits[0]  # Get the hits for the first query

  cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
  cross_scores = cross_encoder.predict(cross_inp)

  # Sort results by the cross-encoder scores
  for idx in range(len(cross_scores)):
      hits[idx]['cross-score'] = cross_scores[idx]

  hits = sorted(hits, key=lambda x: x['score'], reverse=True)

  hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
  hit_passages = []
  hit_cross_scores = []
  for hit in hits[0:n_count]:
    hit_cross_scores.append(hit['cross-score'])
    hit_passages.append(passages[hit['corpus_id']].replace("\n", " "))

  return hit_cross_scores, hit_passages


In [ ]:
def get_result(query, passages, corpus_embeddings, n_count=2):
  hit_scores, hit_passages = retrieve_knowledge(query, passages, corpus_embeddings, n_count=2)
  hit_knowledge = " ".join(hit_passages)
  embeddings1 = similarity_model.encode(ground_knowledge, convert_to_tensor=True)
  embeddings2 = similarity_model.encode(hit_knowledge, convert_to_tensor=True)
  cosine_scores = util.cos_sim(embeddings1, embeddings2)
  return hit_knowledge, cosine_scores[0].item()

In [ ]:
%%capture

bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 1024     #Truncate long passages to 1024 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

In [ ]:
last_dialogID = None
dialogID_index = -1
flag = False
corpus_embeddings = None
results = []

for idx, row in enumerate(tqdm(df.iterrows())):
  row = row[1]
  dialogID = row['dialogID']
  query_rewritten = row['question_rewritten']
  query= row['query']
  utterance = row['utterance']
  ground_knowledge = row['ground_knowledge']

  if dialogID != last_dialogID:
    dialogID_index +=1
    last_dialogID = dialogID
    flag = True

  assert dialogID == test_data[dialogID_index]['dialogID']

  knowledge = "\n".join(test_data[dialogID_index]['knowledge'])
  persona_candidates = test_data[dialogID_index]['utterance'][utterance]['persona_candidate']
  persona_grounding = test_data[dialogID_index]['utterance'][utterance]['persona_grounding']

  paragraphs = get_paragraphs(knowledge)
  passages = get_passages(paragraphs, window_size=1)

  if flag == True:
    try:
      corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True)
    except Exception as e:
      print(f" Error Raised: DialogId: {dialogID} \nException: {e}" )
      continue
  hit_knowledge_0, score_0 = get_result(query, passages, corpus_embeddings, n_count=1)
  hit_knowledge_1, score_1 = get_result(query_rewritten, passages, corpus_embeddings, n_count=1)

  if score_0 > score_1:
    query = query
    hit_knowledge = hit_knowledge_0
    score = score_0
  else:
    query = query_rewritten
    hit_knowledge = hit_knowledge_1
    score = score_1

  results.append({
        "dialogID": dialogID,
        "utterance": utterance,
        "query": query,
        "hit_knowledge": hit_knowledge,
        "ground_knowledge": ground_knowledge,
        "persona_candidates": persona_candidates,
        "persona_grounding": persona_grounding,
        "similarity_score": score
    })

933it [03:29, 14.32it/s]

 Error Raised: DialogId: G7GOCOZDJ39I 
Exception: The size of tensor a (656) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: G7GOCOZDJ39I 
Exception: The size of tensor a (656) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: G7GOCOZDJ39I 
Exception: The size of tensor a (656) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: G7GOCOZDJ39I 
Exception: The size of tensor a (656) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: G7GOCOZDJ39I 
Exception: The size of tensor a (656) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: G7GOCOZDJ39I 
Exception: The size of tensor a (656) must match the size of tensor b (512) at non-singleton dimension 1


13744it [50:55, 12.62it/s]

 Error Raised: DialogId: NZPV3RWXYCAZ 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: NZPV3RWXYCAZ 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: NZPV3RWXYCAZ 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: NZPV3RWXYCAZ 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: NZPV3RWXYCAZ 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: NZPV3RWXYCAZ 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1


33496it [2:03:27,  7.30it/s]

...

 Error Raised: DialogId: 829YXAM13TW4 
Exception: The size of tensor a (521) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: 829YXAM13TW4 
Exception: The size of tensor a (521) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: 829YXAM13TW4 
Exception: The size of tensor a (521) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: 829YXAM13TW4 
Exception: The size of tensor a (521) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: 829YXAM13TW4 
Exception: The size of tensor a (521) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: 829YXAM13TW4 
Exception: The size of tensor a (521) must match the size of tensor b (512) at non-singleton dimension 1


56491it [3:27:40,  9.51it/s]

 Error Raised: DialogId: Z32HKWA20OMR 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: Z32HKWA20OMR 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: Z32HKWA20OMR 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: Z32HKWA20OMR 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: Z32HKWA20OMR 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1
 Error Raised: DialogId: Z32HKWA20OMR 
Exception: The size of tensor a (728) must match the size of tensor b (512) at non-singleton dimension 1

56494it [3:27:40, 13.08it/s]

57928it [3:32:57,  4.53it/s]




*   Window-2 & Count-2 : 0.68034
*   Window-2 & Count-1 : 0.68034
*   Window-1 & Count-1 : 0.682617
*   Window-1 & Count-2 : 0.682617
*   Window-1 & Count-3 : 0.682617




In [ ]:
scores = [result['similarity_score'] for result in results]
sum(scores)/len(scores)

0.6876178949615003

In [ ]:
sum(hit_knowledge_df['similarity_score'].values[:])/len(hit_knowledge_df['similarity_score'].values[:])

0.6021441309675462

In [ ]:
result_df = pd.DataFrame(results)

In [ ]:
result_df.columns

Index(['dialogID', 'utterance', 'query', 'hit_knowledge', 'ground_knowledge',
       'persona_candidates', 'persona_grounding', 'similarity_score'],
      dtype='object')

In [ ]:
folder = "/content/drive/MyDrive/UMBC/knowledge_retrieval"

In [ ]:
result_df.to_csv(f"{folder}/train_query_rewritten_1_1.csv", index = False)

In [ ]:
index =15
print("Query: ", results[index]['query'])

print( results[index]['hit_knowledge'])

Query:  I know this place, but I don't remember the name of this place.
It is the location of the oldest congregation in the city, with the parish being established in 1797. It is located across King Street from St. Lawrence Hall.


In [ ]:
print("Query: ", hit_knowledge_df.iloc[index]['query'])

print(hit_knowledge_df.iloc[index]['hit_knowledge'])

Query:  I know this place, but I don't remember the name of this place.
The Cathedral Church of St. James is an Anglican cathedral in Downtown Toronto, Ontario, Canada. It is the location of the oldest congregation in the city, with the parish being established in 1797.


In [ ]:
hit_knowledge_df.columns

Index(['dialogID', 'utterance', 'query', 'hit_knowledge', 'ground_knowledge',
       'ground_persona', 'similarity_score'],
      dtype='object')

In [ ]:
hit_knowledge_df.head()

,dialogID,utterance,query,hit_knowledge,ground_knowledge,ground_persona,similarity_score
0,CGHGPTDOA0O7,0,I think I've been there before but I don't rem...,Although it is not fully understood how it cam...,Rochdale Town Hall is a Victorian-era municipa...,[],0.564609
1,CGHGPTDOA0O7,1,Where is this place?,Rochdale Town Hall is a Victorian-era municipa...,Rochdale Town Hall is a Victorian-era municipa...,['I would like to go to england.'],0.955529
2,CGHGPTDOA0O7,2,When this was inaugurated?,Built in the Gothic Revival style at a cost of...,Built in the Gothic Revival style at a cost of...,[],0.846570
3,CGHGPTDOA0O7,3,Who was the architect at that time?,Alfred Waterhouse was given the task of design...,"The architect, William Henry Crossland, was th...",['I am not an architect.'],0.382931
4,CGHGPTDOA0O7,4,What was the height of the tower?,The design of the original tower was more elab...,The design of the original tower was more elab...,[],1.000000


In [ ]:
result_df.shape

(57898, 8)